# 01 · Feature Engineering

Pull raw UPRO/SPXU data, validate it, and persist engineered factors for downstream modeling.


**Workflow**
- Ensure `00_environment.ipynb` ran successfully
- Download minute bars (cached under `data/minute_bars.parquet`)
- Compute momentum, volatility, and oscillator features
- Persist the final feature panel to `data/features.parquet`


In [ ]:
from pathlib import Path

import pandas as pd
import alpaca_trade_api as tradeapi

from momentum_lib import (
    bootstrap_env,
    AlpacaDataClient,
    pivot_ohlc,
    compute_features,
)

bootstrap_env(Path("../.env"))
print("Environment primed.")

data_dir = Path("../data")
data_dir.mkdir(exist_ok=True)
cache_file = data_dir / "minute_bars.parquet"


In [ ]:
api = tradeapi.REST()
client = AlpacaDataClient(api=api)
raw = client.fetch_minutes(days=120, cache_file=cache_file)
print(f"Fetched {len(raw):,} rows. Latest timestamp: {raw['ts'].max()}")
raw.head()


In [ ]:
prices = pivot_ohlc(raw)
prices.tail()


In [ ]:
features = compute_features(prices)
feature_file = data_dir / "features.parquet"
features.to_parquet(feature_file)
print(f"Feature panel saved to {feature_file} with shape {features.shape}")
features.describe().T.head()
